In [39]:
#Create lookup table
! python3 /home/teikiet/NuRadioMC/NuRadioReco/modules/neutrinoVertexReconstructor/create_lookup_table.py --output_path /home/teikiet/Data/test_lookup_table/ --ice_model southpole_2015 63

trying to compile the CPP extension on-the-fly
rm: cannot remove 'wrapper.so': No such file or directory
rm: cannot remove 'wrapper.*.so': No such file or directory
/home/teikiet/NuRadioMC/NuRadioMC/SignalProp/install.sh: line 11: python: command not found
compilation was not successful, using python version of ray tracer
check NuRadioMC/NuRadioMC/SignalProp/CPPAnalyticRayTracing for manual compilation
Calculating lookup table for antenna_63.0
/home/teikiet/NuRadioMC/NuRadioMC/SignalProp/analyticraytracing.py:478: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  travel_time = integrate.quad(dt, x1[1], x2_mirrored[1], args=(C_0), points=points, epsabs=1e-10, epsrel=1.49e-08, limit=500)
/home/teikiet/NuRadioMC/NuRadioMC/SignalProp/analyticraytracing.py:260: RuntimeWarning: divide by zero encountered in scalar power
  self.medium.z_0) + self.medium.delta_n ** 2 * np.exp(0.2e1 * z / self.medium.z_0) + c) * c + 0.2e1 * c ** 1.5) / B

In [2]:
import numpy as np
import NuRadioReco.modules.io.eventReader
import NuRadioReco.modules.io.eventWriter
import NuRadioReco.detector.generic_detector
import NuRadioReco.modules.channelResampler
import NuRadioReco.modules.neutrinoVertexReconstructor.neutrino3DVertexReconstructor
import NuRadioReco.modules.channelSignalReconstructor
import NuRadioReco.modules.channelGenericNoiseAdder
from NuRadioReco.utilities import units, bandpass_filter
import NuRadioReco.framework.base_trace
import argparse
"""
parser = argparse.ArgumentParser(
    description='Run the vertex reconstruction used for the RNO-G energy reconstruction'
)
parser.add_argument(
    'lookup_tables',
    type=str,
    help='Folder containing the lookup tables for the vertex reconstructor. To create lookup tables, run '
         'NuRadioReco/modules/neutrinoVertexReconstructor/create_lookup_table.py'
         'You need a lookup table for each antenna depth used in the reconstruction.'
)
parser.add_argument('--input_file', type=str, default='simulated_events.nur', help='File to run the reconstruction on.')
parser.add_argument('--output_file', type=str, default='reconstructed_vertex.nur', help='Filename into which results are written')
parser.add_argument(
    '--detector_file',
    type=str,
    default='../../detector/RNO_G/RNO_single_station.json',
    help='JSON file containing the detector description. Here, we assume it is written for the GenericDetector class.'
)
parser.add_argument('--noise_level', type=float, default=10., help='RMS of the noise to be added, in mV.')
args = parser.parse_args()"""
s_id = 100
input_file = "/home/teikiet/Data/sims_neutrinos_ara2_n1.000e02_4_18to21/Nur/sim_1.000e19.nur"
output_file = 'reconstructed_vertex.nur'
detector_file = "detector/ara2.json"
lookup_tables = "/home/teikiet/Data/test_lookup_table"
noise_level = 1.0661514487245143e-05 #V
noise_level = noise_level * units.mV
sampling_rate = 1.6 * units.GHz

"""
IDs of the channels to be used for the vertex reconstruction, assuming you used the RNO-G detector description set as
the default. The shorter list saves time, but results may be less accurate.
"""
# vertex_channel_ids = [0, 1, 6, 7, 8, 9, 21]
vertex_channel_ids = [0]
"""
Passband of the filter that is applied to the channels for the vertex reconstruction.
"""
vertex_reco_passband = [.1, .3]

"""
Set up modules and detector class
"""
signal_reconstructor = NuRadioReco.modules.channelSignalReconstructor.channelSignalReconstructor()
event_reader = NuRadioReco.modules.io.eventReader.eventReader()
event_reader.begin([input_file])
channel_resampler = NuRadioReco.modules.channelResampler.channelResampler()
event_writer = NuRadioReco.modules.io.eventWriter.eventWriter()
event_writer.begin(output_file)
noise_adder = NuRadioReco.modules.channelGenericNoiseAdder.channelGenericNoiseAdder()
det = NuRadioReco.detector.generic_detector.GenericDetector(
    json_filename=detector_file,
    antenna_by_depth=False
)

"""
We create an electric field template to be used when calculating the timing difference between channels. Pretty much any
short pulse will do, so we create a delta pulse by setting a constant spectrum, apply a filter, and shift the pulse to
be in the middle of the trace.
"""
spec = np.ones(int(128 * sampling_rate + 1)) * bandpass_filter.get_filter_response(
    np.fft.rfftfreq(int(256 * sampling_rate), 1. / sampling_rate), vertex_reco_passband, 'butter', 10)
efield_template = NuRadioReco.framework.base_trace.BaseTrace()
efield_template.set_frequency_spectrum(spec, sampling_rate)
efield_template.apply_time_shift(20. * units.ns, True)

"""
Set up vertex reconstruction modules
"""
vertex_reconstructor = NuRadioReco.modules.neutrinoVertexReconstructor.neutrino3DVertexReconstructor.neutrino3DVertexReconstructor(
    lookup_table_location=lookup_tables
)
"""
These settings are a compromise between accuracy and saving time. Reduce grid size for better results. But be careful
when creating plots: A very fine grid can cause memory problems for matplotlib.
"""
vertex_reconstructor.begin(
    station_id= s_id,
    channel_ids=vertex_channel_ids,
    detector=det,
    template=efield_template,
    distances_2d=np.arange(100, 3600, 200),
    distance_step_3d=10,
    z_step_3d=10,
    widths_3d=np.arange(-50, 50, 10),
    passband=vertex_reco_passband,
    z_coordinates_2d=np.arange(-3000, -50, 2),
    debug_folder='plots/vertex_reco'
)

for i_event, event in enumerate(event_reader.run()):
    print('Event {}, ID={}, Run={}'.format(i_event, event.get_id(), event.get_run_number()))
    station = event.get_station(100)
    station.set_is_neutrino()
    noise_adder.run(event, station, det, amplitude=noise_level, type='rayleigh')
    channel_resampler.run(event, station, det, sampling_rate=sampling_rate)
    vertex_reconstructor.run(
        event,
        station,
        det,
        debug=True
    )
    channel_resampler.run(event, station, det, sampling_rate=2.)
    signal_reconstructor.run(event, station, det)
    event_writer.run(event)

DEBUG:vertexReco:Using 1 channels / 0 channel pairs


Event 0, ID=0, Run=13


SystemError: tile cannot extend outside image

<Figure size 1200x0 with 0 Axes>

In [33]:
import ROOT 
import numpy as np
import matplotlib.pyplot as plt
import radiotools.helper
import scipy
import re
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import h5py
import glob 
from scipy import interpolate
import json
import os
import sys
from scipy import stats
from NuRadioReco.utilities import units
from NuRadioReco.detector import detector
from NuRadioMC.utilities import fluxes
from NuRadioMC.utilities.Veff import get_Veff_Aeff, get_Veff_Aeff_array, get_index, get_Veff_water_equivalent
#from my_Veff import get_Veff_Aeff, get_Veff_Aeff_array, get_index, get_Veff_water_equivalent
from NuRadioMC.examples.Sensitivities import E2_fluxes3 as limits
from Get_hdf5_data import get_attr_data, get_shower_data, get_station_data, get_Veff_data, get_event_data, get_nur_data
from Peak_Finding import count_peak, get_noise, save_data_2_csv, read_column_csv, get_peak, make_max_array
import numpy as np
import matplotlib.pyplot as plt
from NuRadioReco.utilities import units
import NuRadioReco.detector.detector as detector
import NuRadioReco.modules.io.eventReader
import astropy
from NuRadioReco.framework.parameters import stationParameters as stnp
import pandas as pd
from math import isclose
import mplhep as hep
from scipy.signal import find_peaks

PATH = "/home/teikiet/Data/"
##########################################################################
S_ID = [100, 2, 3, 4, 5] #station ID
C_ID = list(range(0,16)) #channel ID
V_pole_channel = list(range(0,8)) #channel ID
H_pole_channel = list(range(8,16)) #channel ID
##########################################################################
E = 18 #energy
detectordescription = "detector/ara2.json" #detector 
##########################################################################


def correlation(channel1, channel2):
    return radiotools.helper.get_normalized_xcorr(channel1,channel2)
def correlation2(trace1, trace2):
    return scipy.signal.correlate(trace1, trace2, mode='full', method='auto') / (np.sum(trace1 ** 2) * np.sum(trace2 ** 2)) ** 0.5

# Open the ROOT file for reag
file1 = "/home/teikiet/Data/grA_ev1_A4_2018_ie0_4096_T26Mar23-131515.979900_run4402.root"
file2 = "/home/teikiet/Data/a.root"
file_A3 = "/home/teikiet/Data/fBC_TGraphs_A3.root"
file = file2
# Loop through the keys and identify TGraph objects

def get_channel_name(file):
    root_file = ROOT.TFile(file, "READ")
    keys = root_file.GetListOfKeys()
    G_name = []
    for key in keys:
        obj = key.ReadObj()
        if isinstance(obj, ROOT.TGraph):
            graph_name = key.GetName()
            G_name.append(graph_name)

    v_pole_channel_name = []
    h_pole_channel_name = []
    for i in range(len(G_name)):
        test_string = (G_name[i][-2:])
        temp = re.findall(r'\d+', test_string)
        temp = int(temp[0])
        if temp <= 7:
            v_pole_channel_name.append(G_name[i])
        else:
            h_pole_channel_name.append(G_name[i])
    return v_pole_channel_name, h_pole_channel_name
v_pole_channel_name, h_pole_channel_name =get_channel_name(file)
print(v_pole_channel_name)
print(h_pole_channel_name)
#Channel info for root file: ############################################################################################################
#v_pole_channel_name = ['grCPAvg2M883_0', 'grCPAvgN_M128_0', 'grfftCPAvgN_M128_0', 'grCPAvg2M883_1', 'grCPAvgN_M128_1', 'grfftCPAvgN_M128_1', 'grCPAvg2M883_2', 'grCPAvgN_M128_2', 'grfftCPAvgN_M128_2', 'grCPAvg2M883_3', 'grCPAvgN_M128_3', 'grfftCPAvgN_M128_3', 'grCPAvg2M883_4', 'grCPAvgN_M128_4', 'grfftCPAvgN_M128_4', 'grCPAvg2M883_5', 'grCPAvgN_M128_5', 'grfftCPAvgN_M128_5', 'grCPAvg2M883_6', 'grCPAvgN_M128_6', 'grfftCPAvgN_M128_6', 'grCPAvg2M883_7', 'grCPAvgN_M128_7', 'grfftCPAvgN_M128_7']
#h_pole_channel_name = ['grCPAvg2M883_8', 'grCPAvgN_M128_8', 'grfftCPAvgN_M128_8', 'grCPAvg2M883_9', 'grCPAvgN_M128_9', 'grfftCPAvgN_M128_9', 'grCPAvg2M883_10', 'grCPAvgN_M128_10', 'grfftCPAvgN_M128_10', 'grCPAvg2M883_11', 'grCPAvgN_M128_11', 'grfftCPAvgN_M128_11', 'grCPAvg2M883_12', 'grCPAvgN_M128_12', 'grfftCPAvgN_M128_12', 'grCPAvg2M883_13', 'grCPAvgN_M128_13', 'grfftCPAvgN_M128_13', 'grCPAvg2M883_14', 'grCPAvgN_M128_14', 'grfftCPAvgN_M128_14', 'grCPAvg2M883_15', 'grCPAvgN_M128_15', 'grfftCPAvgN_M128_15']

def read_root_file(root_file, graph_name):
    root_file = ROOT.TFile(root_file, "READ")
    # Get the TGraph by name
    graph = root_file.Get(graph_name)
    test_string = (graph_name[-2:])
    temp = re.findall(r'\d+', test_string)
    channel_id = int(temp[0])

    test_string = (graph_name[0:9])
    event_id = test_string
    # Check if the TGraph exists
    if graph:
        # Access information about the TGraph
        n_points = graph.GetN()

        # Convert LowLevelViews to Python lists
        #x_values = np.frombuffer(graph.GetX(), dtype=np.float64)
        y_values = np.frombuffer(graph.GetY(), dtype=np.float64)
        return event_id, channel_id, y_values
    else:
        print("TGraph", graph_name, "not found in the ROOT file.")
    root_file.Close()
# Close the ROOT file when done

#######################################################################################################################################
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
#Channel info: #############################################################################################################
v_pole_channel_info = [S_ID, V_pole_channel]
h_pole_channel_info = [S_ID, H_pole_channel]

#Get data:
def get_trigger_event(data, s_ID, c_ID):
    data_frame = pd.read_csv(data)
    event_id = np.array(data_frame["event"])
    trace = np.array(data_frame["Traces"])
    CHNL = np.array(data_frame["channel"])
    STN = np.array(data_frame["station"])
    noise = 1.0661514487245143e-05
    threshold = 4 #sigma
    #0.625 ns
    #num_of_events = int(event_id[-1])
    DATA = []
    ch = CHNL[0]
    st = STN[0]
    evnt = event_id[0]
    i = 0
    trace_i = []
    while i < len(trace):
        if CHNL[i] == ch and STN[i] == st and event_id[i] == evnt:
            if CHNL[i] in c_ID and STN[i] in s_ID:
                trace_i.append(float(trace[i]))
            else:
                pass
        else:
            if len(trace_i) > 0 and sum(trace_i) != 0:
                snr_i = abs(np.array(trace_i))/noise
                num_bin = int(len(snr_i)/11) #11 is the bin size
                snr_i = make_max_array(snr_i, num_bin)
                peaks, _ = find_peaks(snr_i, height=threshold)
                num_peak = len(peaks)
                data_i = []
                data_i.append(num_peak)
                data_i.append(st)
                data_i.append(ch)
                data_i.append(trace_i)
                data_i.append(evnt)
                DATA.append(data_i)
                trace_i = []
            ch = CHNL[i]
            st = STN[i]
            evnt = event_id[i]
        i += 1
    return DATA

def get_trace_from_root(root_file, G_name):
    data = []
    event_id = 1
    event_name = read_root_file(root_file, G_name[0])[0]
    for i in range(len(G_name)):
        data_i = []
        if event_name != read_root_file(root_file, G_name[i])[0]:
            event_id += 1
            event_name = read_root_file(root_file, G_name[i])[0]
        channel_name = read_root_file(root_file, G_name[i])[1]
        trace = read_root_file(root_file, G_name[i])[2]
        data_i.append(event_id)
        data_i.append(channel_name)
        data_i.append(trace)
        data.append(data_i)
    return data
#Get trace:
def get_trace_from_data(data_neu, data_mm, data_noise, channel_info):
    trace_neu =  get_trigger_event(data_neu, *channel_info)
    trace_mm =  get_trigger_event(data_mm, *channel_info)
    trace_noise =  get_trigger_event(data_noise, *channel_info)
    return trace_neu, trace_mm, trace_noise

def RPR(trace):
    num_subarrays = int(len(trace)/40)
    subarrays = np.array_split(trace, num_subarrays)
    power_values = np.array([np.sqrt(1/len(subarray) * sum(subarray**2)) for subarray in subarrays])
    max_power = max(power_values)
    index = np.where(power_values != max_power)
    power_values = power_values[index]
    rms = np.sqrt(sum(power_values**2)/len(power_values))
    return max_power/rms

def get_correlation_between_channel(data_trace_1, data_trace_2):
    event = []
    rpr = []
    corr = []
    if data_trace_1 == data_trace_2 and len(data_trace_1[0]) == 5:
        for i in range(len(data_trace_1)):
            for j in range(i+1, len(data_trace_2)):
                event1 = data_trace_1[i][4]
                event2 = data_trace_2[j][4]

                station1 = data_trace_1[i][1]
                station2 = data_trace_2[j][1]
                
                if station1 == station2 and event1 == event2:
                    trace_1 = data_trace_1[i][3]
                    trace_2 = data_trace_2[j][3]
                    if sum(trace_1) != 0 and sum(trace_2) != 0:
                        event_i = event1
                        rpr_i = max(RPR(trace_1), RPR(trace_2))
                        corr_i =max(correlation(np.array(trace_1), np.array(trace_2)))
                        event.append(event_i)
                        rpr.append(rpr_i)
                        corr.append(corr_i)
    elif len(data_trace_1[0]) == 3:
        for i in range(len(data_trace_1)):
            for j in range(i+1, len(data_trace_2)):
                event_i = data_trace_1[i][0]
                channel1 = data_trace_1[i][1]
                channel2 = data_trace_2[j][1]
                if channel1 != channel2:
                    trace_1 = data_trace_1[i][2]
                    trace_2 = data_trace_2[j][2]
                    event.append(event_i)
                    rpr_i = max(RPR(trace_1), RPR(trace_2))
                    corr_i = max(correlation(np.array(trace_1), np.array(trace_2)))
                    rpr.append(rpr_i)
                    corr.append(corr_i)
    return event, rpr, corr
            

def get_pair_correlation(trace_1, trace_2, peak1, peak2):
    if trace_1 == trace_2:
        p = [max(correlation(np.array(trace_1[i][3]), np.array(trace_2[j][3])))
                                for i in range(len(trace_1))
                                for j in range(i+1, len(trace_2))
                                if trace_1[i][1] == trace_2[j][1] and trace_1[i][0] in peak1 and trace_2[j][0] in peak2] #remove duplicate
    elif trace_1 != trace_2 and type(trace_1[0]) != str:
        p = [max(correlation(np.array(trace_1[i][3]), np.array(trace_2[j][3])))
                                for i in range(len(trace_1))
                                for j in range(len(trace_2))
                                if trace_1[i][1] == trace_2[j][1] and trace_1[i][0] in peak1 and trace_2[j][0] in peak2]
    elif type(trace_1[0]) == str:
        p = [max(correlation(read_root_file(file, trace_1[i])[1], np.array(trace_2[j][3])))
                            for i in range(len(trace_1))
                            for j in range(len(trace_2))
                            if read_root_file(file, trace_1[i])[0] == trace_2[j][2] and trace_2[j][0] in peak2]
    p = np.array(p)
    p = p[~np.isnan(p)] #remove nan
    return p

def get_correlation(trace_neu, trace_mm, trace_noise, G_name, peak1, peak2):
    #neutrino to neutrino
    pairs_correlation_n2n = get_pair_correlation(trace_neu, trace_neu, peak1, peak2)
    #Monopole to monopole
    pairs_correlation_m2m = get_pair_correlation(trace_mm, trace_mm, peak1, peak2)
    #Neutrino to monopole
    pairs_correlation_m2n = get_pair_correlation(trace_mm, trace_neu, peak1, peak2)
    #Cal Pulser to neutrino
    pairs_correlation_c2n = get_pair_correlation(G_name, trace_neu, peak1, peak2)
    #Cal Pulser to monopole
    pairs_correlation_c2m = get_pair_correlation(G_name, trace_mm, peak1, peak2)
    #noise to cal pulser:
    pairs_correlation_noise2c = get_pair_correlation(G_name, trace_noise, peak1, peak2)
    return pairs_correlation_n2n, pairs_correlation_m2m, pairs_correlation_m2n, pairs_correlation_c2n, pairs_correlation_c2m, pairs_correlation_noise2c

def get_list_peak(trace_data):
    P = [trace_data[i][0] for i in range(len(trace_data))]
    max_P = max(P)
    list_P = np.arange(0, max_P+1)
    list_P = list(list_P)
    return list_P

def get_mean_error(event, rpr, corr):
    mean_CORR = []
    err_CORR = []
    mean_RPR = []
    err_RPR = []

    EVENT = []

    corr_i = []
    rpr_i = []
    event_i = event[0]
    i = 0
    while i < len(event):
        if event[i] == event_i:
            event_i = event[i]
            corr_i.append(corr[i])
            rpr_i.append(rpr[i])
        else:
            mean_CORR.append(np.mean(corr_i))
            err_CORR.append(stats.sem(corr_i))
            mean_RPR.append(np.mean(rpr_i))
            err_RPR.append(stats.sem(rpr_i))
            EVENT.append(event_i)
            corr_i = []
            rpr_i = []
            event_i = event[i]
        i += 1
    return EVENT, mean_CORR, err_CORR, mean_RPR, err_RPR   
def get_efficiency(event, corr, rpr, threshold_rpr, threshold_corr):
    event = np.array(event)
    corr = np.array(corr)
    rpr = np.array(rpr)
    index = np.where(corr >= threshold_corr)
    
    event_pass = event[index]
    rpr = rpr[index]
    index = np.where(rpr >= threshold_rpr)
    event_pass = event_pass[index]
    return len(event_pass)/len(corr)

['grEv27185Ch0', 'grEv27185Ch1', 'grEv27185Ch2', 'grEv27185Ch3', 'grEv27185Ch4', 'grEv27185Ch5', 'grEv27185Ch6', 'grEv27185Ch7', 'grEv27235Ch0', 'grEv27235Ch1', 'grEv27235Ch2', 'grEv27235Ch3', 'grEv27235Ch4', 'grEv27235Ch5', 'grEv27235Ch6', 'grEv27235Ch7', 'grEv27237Ch0', 'grEv27237Ch1', 'grEv27237Ch2', 'grEv27237Ch3', 'grEv27237Ch4', 'grEv27237Ch5', 'grEv27237Ch6', 'grEv27237Ch7', 'grEv27260Ch0', 'grEv27260Ch1', 'grEv27260Ch2', 'grEv27260Ch3', 'grEv27260Ch4', 'grEv27260Ch5', 'grEv27260Ch6', 'grEv27260Ch7', 'grEv27269Ch0', 'grEv27269Ch1', 'grEv27269Ch2', 'grEv27269Ch3', 'grEv27269Ch4', 'grEv27269Ch5', 'grEv27269Ch6', 'grEv27269Ch7', 'grEv27299Ch0', 'grEv27299Ch1', 'grEv27299Ch2', 'grEv27299Ch3', 'grEv27299Ch4', 'grEv27299Ch5', 'grEv27299Ch6', 'grEv27299Ch7', 'grEv27317Ch0', 'grEv27317Ch1', 'grEv27317Ch2', 'grEv27317Ch3', 'grEv27317Ch4', 'grEv27317Ch5', 'grEv27317Ch6', 'grEv27317Ch7', 'grEv27318Ch0', 'grEv27318Ch1', 'grEv27318Ch2', 'grEv27318Ch3', 'grEv27318Ch4', 'grEv27318Ch5', 'grEv27

In [19]:
# data in csv file:
file_8074 = "/home/teikiet/Data/fBC_TGraphs_run8074.root"
file_a = "/home/teikiet/Data/a.root"
#file = "/home/teikiet/Data/grA_ev1_A4_2018_ie0_4096_T26Mar23-131515.979900_run4402.root"
def corr_rpr(file):
    file_a = file
    v_pole_channel_name_a, h_pole_channel_name_a = get_channel_name(file_a)
    data_a = get_trace_from_root(file_a, v_pole_channel_name_a)
    event_a, rpr_a, corr_a = get_correlation_between_channel(data_a, data_a)
    EVENT_a, mean_CORR_a, err_CORR_a, mean_RPR_a, err_RPR_a = get_mean_error(event_a, rpr_a, corr_a)
    return EVENT_a, mean_CORR_a, err_CORR_a, mean_RPR_a, err_RPR_a

################################
corr_rpr(file_8074)

([1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26],
 [0.15797898808342814,
  0.14834524541239608,
  0.1484641599973565,
  0.16696487614339334,
  0.17094485834458886,
  0.170476852574645,
  0.16721815665484535,
  0.17064290741404595,
  0.17017781645714808,
  0.19518499730022923,
  0.17787947002639076,
  0.181676847624018,
  0.16184205347916547,
  0.20811062111555842,
  0.1354742551590218,
  0.1793224853585859,
  0.17868633366928013,
  0.17055778949955527,
  0.16908875722201308,
  0.15838381118610015,
  0.1550343792065143,
  0.15705300855454582,
  0.16784479925483878,
  0.15540482907189473,
  0.20316082347889114,
  0.16576186966760276],
 [0.0008117636212723117,
  0.00043869318077678057,
  0.0004847360665652718,
  0.000700391548712159,
  0.000826387898569948,
  0.0011392127736854292,
  0.0007272922997193997,
  0.0008419046291483897,
  0.0007650576110675063,
  0.0010402164811197788,
  0.00090374

In [26]:
#get file size in bytes:
file_A3 = "/home/teikiet/Data/fBC_TGraphs_A3.root"
def get_file_size(file):
    file_size = os.path.getsize(file)
    return file_size
t = [36.5, 228]
print(get_file_size(file_a)/ get_file_size(file_8074), get_file_size(file_A3)/get_file_size(file_a))

2.588439672783184 53.028090193478775


In [32]:
(128 * 228)/(60*60)

8.106666666666667